In [1]:
from bs4 import BeautifulSoup
import pandas as pd

apple_file = "./data-2/apple/Apple_Full text.XML"

with open(apple_file, 'r') as f:
    data = f.read()

soup = BeautifulSoup(data, "html.parser")
apple_case = soup.findAll("doc")
result = []
for i in apple_case:
    uid = i.find("uuid").text
    content1 = i.find("str", {"name": "權利要求"})
    content2 = i.find("str", {"name": "說明書"})
    content3 = i.find("str", {"name": "公開(公告)號"})

    content1 = content1.text if content1 is not None else ""
    content2 = content2.text if content2 is not None else ""
    content3 = content3.text if content3 is not None else ""

    tmp_df = pd.DataFrame({
        "uid": [uid],
        "公開號": [content3],
        "content": [content1+" "+content2]
    })
    result.append(tmp_df)

result = pd.concat(result)
# result.to_excel(self.apple_data, index=False)

c:\Users\cglog\miniconda3\envs\torch\lib\site-packages\bs4\builder\__init__.py:546: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  XMLParsedAsHTMLWarning.MESSAGE, XMLParsedAsHTMLWarning


In [2]:
result.head(3)

,uid,公開號,content
0,9cf5edf8-532c-443c-9881-8b415d5efa54,US9668676B2,1. A method of an electronic device including ...
0,f894d5f0-874c-47fe-b212-b901d0e188ea,US10517489B2,1. A method for determining a pressure of bloo...
0,eef31886-7293-4a41-98ca-10729319249b,US10620232B2,"1. A method comprising:\nreceiving, by a weara..."


In [3]:
result.to_excel("./data-2/apple.xlsx", index=False)

In [ ]:
from Task1 import Data

data_info = {
    "other_data": "./data-2/other.xlsx",
    "other_folder": "./data-2/other/*",
    "apple_data": "./蘋果專利.xlsx",
    "apple_file": "./data-2/apple/Apple_Full text.XML",
}

apple_df = Data(**data_info).apple()
other_df = Data(**data_info).other()

len(apple_df), len(other_df)

In [ ]:
apple_df.info()

In [2]:
other_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60729 entries, 0 to 65277
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Country Code                  60729 non-null  object
 1   Granted Number                60729 non-null  object
 2   Examined Pub. Number          60729 non-null  object
 3   Unexamined Pub. Number        60729 non-null  object
 4   Kind Code                     60729 non-null  object
 5   Application Number            60729 non-null  object
 6   Application Date              60729 non-null  object
 7   Unexamined Pub. Date          60729 non-null  object
 8   Examined Pub. Date            60729 non-null  object
 9   Status[US,JP,KR,CN,EP,CA,AU]  60729 non-null  object
 10  Applicant                     60729 non-null  object
 11  Orig. IPC(All)                60729 non-null  object
 12  Curr. IPC(All)                60729 non-null  object
 13  Title           

# 公司歸屬

In [ ]:
## 驗證
import re
def company_v(company:str):
    company = re.sub(r"[^a-zA-Z0-9]", "", company)
    company = company.capitalize()
    return company
other_df['company_verify'] = other_df['Applicant'].map(company_v)

In [10]:
from tqdm import tqdm
Applicant_set  = other_df['Applicant_verify'].unique()
associate_code = {}
for i in tqdm(other_df['Applicant_verify']):
    for i2 in Applicant_set:
        if i in i2:
            if associate_code.get(i) is None:
                associate_code[i] = [i2]
            else:
                associate_code[i].extend([i2])

100%|██████████| 60729/60729 [02:20<00:00, 432.87it/s]


In [15]:
for k in associate_code.keys():
    associate_code[k] = list(set(associate_code[k]))

In [18]:
associate_code
new_applicant_code = {index: k for index, k in enumerate(associate_code.keys())}

In [21]:
for applicant_code, applicant_name in new_applicant_code.items():
    new_applicant_code[applicant_code] = [applicant_name]
    new_applicant_code[applicant_code].extend(associate_code[applicant_name])

In [23]:
def give_new_applicant_id(applicant_verify):
    for code, name in new_applicant_code.items():
        if applicant_verify in name:
            return code
    return -1

other_df['new_applicant_id'] = other_df['Applicant_verify'].map(give_new_applicant_id)

In [42]:
other_df_process_company_count = other_df.groupby(by=['new_applicant_id', 'Applicant']).count().reset_index().loc[:,["new_applicant_id", "Applicant", "Country Code"]].rename({"Country Code": "count"}, axis=1)
other_df_process_company_count = other_df_process_company_count.sort_values(by=['new_applicant_id', 'count'])
other_df_process_company_count.to_excel("公司專利數量(特殊處理).xlsx")

In [49]:
other_df.drop(labels=['Applicant Id', 'Applicant_verify'], axis=1).to_excel("./公司專利(公司名已處理).xlsx")

# 數據驗證

In [1]:
import pandas as pd

apple_df = pd.read_excel("./data-2/apple.xlsx")
len(apple_df)

352

In [2]:
apple_df.head()

,uid,content
0,9cf5edf8-532c-443c-9881-8b415d5efa54,1. A method of an electronic device including ...
1,f894d5f0-874c-47fe-b212-b901d0e188ea,1. A method for determining a pressure of bloo...
2,eef31886-7293-4a41-98ca-10729319249b,"1. A method comprising:\nreceiving, by a weara..."
3,a2ba1eea-1d7f-40db-bfd2-2bede2ba31a8,1. A method of adjusting a fit of a watch band...
4,2da9069d-8d4e-4ca8-bd85-61e2f55f52a6,"1. A method comprising:\nreceiving, into one o..."


In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("anferico/bert-for-patents")

c:\Users\cglog\miniconda3\envs\project_similar\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\cglog\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [5]:
len(tokenizer.encode(apple_df.content.iloc[0]))

6321

In [8]:
from PatentModel import PatentModel

model = PatentModel(use_gpu=False)

Some weights of the model checkpoint at anferico/bert-for-patents were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
emb = model.embedding_by_concat(apple_df.content.iloc[0])

In [10]:
emb.shape

(1024,)

In [7]:
apple_df.content.iloc[0]

"1. A method of an electronic device including a plurality of light sensors and light emitters, the method comprising:\nreceiving first light information at a first light sensor co-located with a first light emitter of at least a first wavelength, the first light information based on a reflection of light emitted by the first light emitter, wherein receiving the first light information includes receiving a plurality of first light information data points over a plurality of collection times;\nreceiving second light information at a second light sensor co-located with a second light emitter of at least a second wavelength, different from the first wavelength, the second light information based on a reflection of light emitted by the second light emitter, the second light sensor and the second light emitter separate and distinct from the first light sensor and the first light emitter, respectively,\nwherein receiving the second light information includes receiving a plurality of second l

In [11]:
other_df

NameError: name 'other_df' is not defined

In [1]:
import numpy as np

In [10]:
from tqdm import tqdm
from glob import glob

g = None
n= 60279
for i in tqdm(range(n)):
    f = r"C:\Users\cglog\Desktop\project-nlp-patent-similarity\concat-20230323T163053Z-001\concat\{}.npy".format(i)
    tmp = np.load(f)
    if g is None:
        g = tmp
    else:
        g = np.concatenate((g, tmp), axis=0)


100%|██████████| 60279/60279 [6:13:58<00:00,  2.69it/s]       


In [11]:
np.save("concat.npy", g)

In [2]:
import numpy as np
from tqdm import tqdm
from glob import glob

g = None
n= 60279
for i in tqdm(range(n)):
    f = r"C:\Users\cglog\Desktop\project-nlp-patent-similarity\overlapping\{}.npy".format(i)
    f = r"C:\Users\cglog\Desktop\project-nlp-patent-similarity\overlapping\{}.npy".format(i)
    tmp = np.load(f)
    if g is None:
        g = tmp
    else:
        g = np.concatenate((g, tmp), axis=0)


100%|██████████| 60279/60279 [41:25<00:00, 24.26it/s]  


In [3]:
np.save("overlapping.npy", g)